In [1]:
import os
import glob
import pandas as pd
import numpy as np

In [2]:
# Get the name of all 100 clips into a list

clip_path = "/Users/f004p74/Documents/dartmouth/projects/NNDb/movie_clips/"
clip_dir = os.listdir(clip_path)
movie_list = [x for x in clip_dir if '.' not in x] #generates a list of all 10 movies
clip_list = []

zip_name, zip_start, zip_stop = [],[],[]


for movie in movie_list:
    clips = [x for x in os.listdir(clip_path+movie) if 'mp4' in x]
    
    for clip in clips:
        name = clip.split('.')[0]
        start = name.split('_')[1]
        stop = name.split('_')[2]
        clip_list.append(name)

        zip_name.append(name)
        zip_start.append(start)
        zip_stop.append(stop)


In [35]:
clip_dict = dict(zip(zip_name, zip(zip_start, zip_stop)))


In [27]:
# Get a list of all the participant .nii files

img_path = "/Users/f004p74/Documents/dartmouth/projects/NNDb/data_fmri/"
out_path = "/Users/f004p74/Documents/dartmouth/projects/NNDb/analyses/trimmed_fmri/"
func_data = os.listdir(img_path)
sub_data = [x for x in func_data if ('sub-') in x] #grab all the subject IDs for easy filtering

#make a sub-directory for each subject's trimmed nii files
# for sub in sub_data:
#     sub_id = sub.split('_')[0]
#     os.mkdir(out_path+sub_id)

# len(sub_data) # 86


In [33]:
for movie in movie_list[0:1]:
    sub_clips = [x for x in clip_list if movie in x]
    sub_subs =  [x for x in sub_data if movie in x]

    for sub in sub_subs[0:1]:
        sub_id = sub.split('_')[0]
        img = nb.load(img_path+sub)
        for clip in sub_clips[0:1]:
            start = int(clip_dict[clip][0]) - 4
            stop = int(clip_dict[clip][1]) + 4
            #clip_slice = img.slicer[:,:,:,start:stop]

            fname = sub_id+os.sep+sub_id+"_"+movie+"_"+str(start)+"_"+str(stop)
            nb.save(clip_slice, out_path+fname+".nii.gz")


## Apply schaefer atlas to clipped nii files

In [2]:
import nibabel as nb
import nilearn
from nilearn import datasets
from nilearn import plotting
import nilearn.image as image
from nilearn.maskers import NiftiMasker

In [3]:
schaefer_atlas = datasets.fetch_atlas_schaefer_2018(n_rois=100, yeo_networks=17, resolution_mm=1,
                                                    data_dir=None, base_url=None, resume=True, verbose=1)
'''
From the documentation:
The list of labels does not contain ‘Background’ by default. 
To have proper indexing, you should either manually add ‘Background’ to the list of labels:
'''

schaefer_atlas.labels = np.insert(schaefer_atlas.labels, 0, "Background")

In [4]:
clip_path = "/Users/f004p74/Documents/dartmouth/projects/NNDb/analyses/trimmed_fmri/sub-71/sub-71_split_5605_5637.nii.gz"
clip_slice = nb.load(clip_path)

In [5]:
clip_avg = image.mean_img(clip_slice) #shape is (64, 76, 64)

In [6]:
for i in range(1,5): #start with 1 to ignore the background.
    parcel = nilearn.image.new_img_like(schaefer_atlas.maps, nilearn.image.get_data(schaefer_atlas.maps) == i)
    masker = NiftiMasker()
    parcel_mask = masker.fit(parcel)
    roi_data = parcel_mask.transform_single_imgs(clip_avg)
    print(roi_data,roi_data.shape)
    

/opt/anaconda3/lib/python3.11/site-packages/nilearn/maskers/nifti_masker.py:110: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(


[[-0.2238211  -0.14392954 -0.2006803  ... -2.6160626  -4.0625095
  -2.814539  ]] (1, 9158)


/opt/anaconda3/lib/python3.11/site-packages/nilearn/maskers/nifti_masker.py:110: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(


[[ 0.2397058   0.30108017  0.33360443 ... -1.3313439  -1.2829909
  -1.2636914 ]] (1, 12663)


/opt/anaconda3/lib/python3.11/site-packages/nilearn/maskers/nifti_masker.py:110: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(


[[-0.52525437 -0.3709902  -0.16731267 ... -0.00613056 -0.00199834
  -0.01979208]] (1, 7576)


/opt/anaconda3/lib/python3.11/site-packages/nilearn/maskers/nifti_masker.py:110: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(


[[0.12065879 0.17848925 0.1699716  ... 0.42132995 0.37926275 0.37456697]] (1, 14834)
